In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import os
import win32com.client as win32
from datetime import datetime

# ★★★ openpyxl 임포트 ★★★
import openpyxl
from openpyxl.styles import Alignment, Font, Border, Side

class LibraryApp:
    def __init__(self, root):
        self.root = root
        self.root.title("김제학생교육문화관 도서관일지")

        # 모든 Entry 위젯을 담아둘 딕셔너리
        self.entries = {}

        # 날짜 선택용 콤보박스 리스트
        self.years = [str(y) for y in range(2020, 2031)]   # 2020 ~ 2030
        self.months = [str(m) for m in range(1, 13)]       # 1 ~ 12
        self.days = [str(d) for d in range(1, 32)]         # 1 ~ 31
        self.weekdays = ["월", "화", "수", "목", "금", "토", "일"]

        # 1) 큰 제목
        self.create_title_section()
        
        # 2) 화면 상단: 날짜 선택 + 첫번째 테이블
        self.create_header_and_table1()

        # 3) 두 번째 표: 관외대출도서
        self.create_table2()

        # 4) 세 번째 표: 자료실 이용자
        self.create_table3()

        # 5) 네 번째 표: 정보봉사(책바다/책나래)
        self.create_table4()

        # 6) 하단 버튼들 (저장, Excel 열기, 초기화, 출력)
        self.create_buttons()

    def create_title_section(self):
        """
        상단에 제목만 표시하는 영역입니다.
        """
        title_frame = ttk.Frame(self.root, padding=10)
        title_frame.pack(fill="x", padx=5, pady=(5, 0))
        
        title_label = ttk.Label(title_frame,
                                text="김제학생교육문화관 도서관일지",
                                font=("맑은 고딕", 18, "bold"),
                                anchor="center")
        title_label.pack(fill="x")
        
        return title_frame
    
    # ─────────────────────────────────────────────────────────────────────
    def create_header_and_table1(self):
        """
        상단 한 행에는 왼쪽에 날짜 선택 위젯, 오른쪽에 미니 테이블(결재/담당자/담당)을 배치하고,
        그 아래에 큰 표(table1)를 전체 폭에 걸쳐 배치하는 함수.
        
        미니 테이블 구성:
          - 첫 행: [결재] (왼쪽, 3행 병합), [담당자], [담당]
          - 두 번째 행: 입력 위젯을 각 열에 배치하고, 이 위젯은 rowspan=2로 병합하여 총 2개의 입력 필드만 사용.
        """
        # ─────────────────────────────
        # 전체 컨테이너
        self.container = ttk.Frame(self.root)
        self.container.pack(fill="both", expand=True, padx=5, pady=5)
        
        # ─────────────────────────────
        # 상단 헤더 행: 날짜 선택 (왼쪽) + 미니 테이블 (오른쪽)
        # ─────────────────────────────
        header_row = ttk.Frame(self.container)
        header_row.pack(fill="x", padx=5, pady=5)
        
        # [1] 왼쪽: 날짜 선택 영역
        date_frame = ttk.Frame(header_row)
        date_frame.pack(side="left", anchor="w")
        ttk.Label(date_frame, text="연:").pack(side=tk.LEFT, padx=2)
        self.year_cmb = ttk.Combobox(date_frame, values=self.years, width=5)
        self.year_cmb.pack(side=tk.LEFT, padx=2)
        current_year_str = str(datetime.now().year)
        if current_year_str in self.years:
            self.year_cmb.current(self.years.index(current_year_str))
        else:
            self.year_cmb.current(0)
        ttk.Label(date_frame, text="월:").pack(side=tk.LEFT, padx=2)
        self.month_cmb = ttk.Combobox(date_frame, values=self.months, width=3)
        self.month_cmb.pack(side=tk.LEFT, padx=2)
        self.month_cmb.current(datetime.now().month - 1)
        ttk.Label(date_frame, text="일:").pack(side=tk.LEFT, padx=2)
        self.day_cmb = ttk.Combobox(date_frame, values=self.days, width=3)
        self.day_cmb.pack(side=tk.LEFT, padx=2)
        self.day_cmb.current(datetime.now().day - 1)
        ttk.Label(date_frame, text="요일:").pack(side=tk.LEFT, padx=2)
        self.weekday_cmb = ttk.Combobox(date_frame, values=self.weekdays, width=3)
        self.weekday_cmb.pack(side=tk.LEFT, padx=2)
        today_weekday = datetime.now().weekday()  # 0=월, 6=일
        self.weekday_cmb.current(today_weekday % 7)
        
        # [2] 오른쪽: 미니 테이블 영역 (결재/담당자/담당)
        mini_table_frame = ttk.Frame(header_row, borderwidth=2, relief="solid")
        mini_table_frame.pack(side="right", anchor="ne", padx=5, pady=5)
        # --- 첫 행: 헤더 ---
        # "결재" : 0행, 0열, rowspan=3 (변경 없음)
        label_approval = ttk.Label(mini_table_frame, text="결재", borderwidth=1, relief="solid", anchor="center")
        label_approval.grid(row=0, column=0, rowspan=3, sticky="nsew", padx=1, pady=1)
        # "담당자" : 0행, 1열
        label_person = ttk.Label(mini_table_frame, text="담당자", borderwidth=1, relief="solid", anchor="center")
        label_person.grid(row=0, column=1, sticky="nsew", padx=1, pady=1)
        # "담당" : 0행, 2열
        label_resp = ttk.Label(mini_table_frame, text="담당", borderwidth=1, relief="solid", anchor="center")
        label_resp.grid(row=0, column=2, sticky="nsew", padx=1, pady=1)
        # --- 두 번째 행: 입력 필드 ---
        # 각 입력 필드는 2행을 차지하도록 rowspan=2와 ipady 옵션을 추가하여 높이를 키웁니다.
        self.person_entry = ttk.Entry(mini_table_frame, width=10)
        self.person_entry.grid(row=1, column=1, rowspan=2, sticky="nsew", padx=1, pady=1, ipady=10)
        self.responsibility_entry = ttk.Entry(mini_table_frame, width=10)
        self.responsibility_entry.grid(row=1, column=2, rowspan=2, sticky="nsew", padx=1, pady=1, ipady=10)

        
        # ─────────────────────────────
        # 아래 영역: 큰 표(table1)
        # ─────────────────────────────
        table_frame = ttk.Frame(self.container, borderwidth=2)
        table_frame.pack(fill="both", padx=5, pady=(0, 0))
        
        # 예시: table1 구성 (열람실·문화관 교육생·개관일수)
        # 왼쪽 병합: "열람실 이용자"
        lbl_열람 = ttk.Label(table_frame, text="열람실 이용자", borderwidth=1, relief="solid", anchor="center")
        lbl_열람.grid(row=0, column=0, rowspan=3, sticky="nsew", padx=1, pady=1)
        # 헤더 행: "구분", "학생", "일반", "합계"
        ttk.Label(table_frame, text="구분", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=1, sticky="nsew", padx=1, pady=1)
        ttk.Label(table_frame, text="학생", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=2, sticky="nsew", padx=1, pady=1)
        ttk.Label(table_frame, text="일반", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=3, sticky="nsew", padx=1, pady=1)
        ttk.Label(table_frame, text="합계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=4, sticky="nsew", padx=1, pady=1)
        # 중간 병합: "문화관 교육생"
        lbl_문화관 = ttk.Label(table_frame, text="문화관 교육생", borderwidth=1, relief="solid", anchor="center")
        lbl_문화관.grid(row=0, column=5, rowspan=3, sticky="nsew", padx=1, pady=1)
        ttk.Label(table_frame, text="학생교육", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=6, sticky="nsew", padx=1, pady=1)
        ttk.Label(table_frame, text="평생교육", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=7, sticky="nsew", padx=1, pady=1)
        ttk.Label(table_frame, text="시설견학", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=8, sticky="nsew", padx=1, pady=1)
        ttk.Label(table_frame, text="합계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=9, sticky="nsew", padx=1, pady=1)
        # 우측 병합: "개관일수"
        lbl_개관 = ttk.Label(table_frame, text="개관일수", borderwidth=1, relief="solid", anchor="center")
        lbl_개관.grid(row=0, column=10, columnspan=2, sticky="nsew", padx=1, pady=1)
        # 금일/누계 라벨
        ttk.Label(table_frame, text="금일", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=1, sticky="nsew", padx=1, pady=1)
        ttk.Label(table_frame, text="누계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=2, column=1, sticky="nsew", padx=1, pady=1)
        
        # 금일(열람실) 입력 필드 (예시)
        keys_gumil_열람 = ["금일_열람_학생", "금일_열람_일반", "금일_열람_합계"]
        for i, key in enumerate(keys_gumil_열람, start=2):
            ent = ttk.Entry(table_frame, width=8)
            ent.grid(row=1, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # 누계(열람실) 입력 필드 (예시)
        keys_nu_열람 = ["누계_열람_학생", "누계_열람_일반", "누계_열람_합계"]
        for i, key in enumerate(keys_nu_열람, start=2):
            ent = ttk.Entry(table_frame, width=8)
            ent.grid(row=2, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # 금일(문화관 교육생) 입력 필드 (예시)
        keys_gumil_문화 = ["금일_문화_학생교육", "금일_문화_평생교육", "금일_문화_시설견학", "금일_문화_합계"]
        for i, key in enumerate(keys_gumil_문화, start=6):
            ent = ttk.Entry(table_frame, width=8)
            ent.grid(row=1, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # 누계(문화관 교육생) 입력 필드 (예시)
        keys_nu_문화 = ["누계_문화_학생교육", "누계_문화_평생교육", "누계_문화_시설견학", "누계_문화_합계"]
        for i, key in enumerate(keys_nu_문화, start=6):
            ent = ttk.Entry(table_frame, width=8)
            ent.grid(row=2, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # 열람실/자료실 개관일수 입력 필드 (예시)
        ttk.Label(table_frame, text="열람실", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=10, sticky="nsew", padx=1, pady=1)
        ent_open1 = ttk.Entry(table_frame, width=8)
        ent_open1.grid(row=1, column=11, sticky="nsew", padx=1, pady=1)
        self.entries["금일_개관_열람실"] = ent_open1
        ttk.Label(table_frame, text="자료실", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=2, column=10, sticky="nsew", padx=1, pady=1)
        ent_open2 = ttk.Entry(table_frame, width=8)
        ent_open2.grid(row=2, column=11, sticky="nsew", padx=1, pady=1)
        self.entries["누계_개관_자료실"] = ent_open2
        
        for c in range(12):
            table_frame.columnconfigure(c, weight=1)


    # ─────────────────────────────────────────────────────────────────────
    # (3) 두 번째 표: 관외대출도서
    # ─────────────────────────────────────────────────────────────────────
    def create_table2(self):
        # LabelFrame 대신 일반 Frame 사용 (text 인자 없음)
        loan_frame = ttk.Frame(self.container)
        loan_frame.pack(padx=10, pady=(0, 5), fill=tk.X)
        
        categories = ["총류", "철학", "종교", "사회과학", "자연과학",
                      "기술과학", "예술", "언어", "문학", "역사", "계"]
        
        # 가장 왼쪽 0열에 "관외 대출 도서" Label 추가 (행 전체 span: 7행(0~6))
        lbl_title = ttk.Label(loan_frame, text="관외 대출 도서", borderwidth=1, relief="solid", anchor="center")
        lbl_title.grid(row=0, column=0, rowspan=7, sticky="nsew", padx=1, pady=1)
        
        # (0) 헤더행: 원래 구분은 0열에 있었으나, 이제 오른쪽으로 한 칸 이동 (열 1~2)
        lbl_gubun = ttk.Label(loan_frame, text="구분", borderwidth=1, relief="solid", anchor="center")
        lbl_gubun.grid(row=0, column=1, columnspan=2, sticky="nsew", padx=1, pady=1)
        
        # 각 카테고리 라벨: 원래 start=2 → 이제 start=3
        for i, cat in enumerate(categories, start=3):
            ttk.Label(loan_frame, text=cat, borderwidth=1, relief="solid", anchor="center")\
                .grid(row=0, column=i, sticky="nsew", padx=1, pady=1)
        
        # (1) 일반자료실: 금일/누계 (행 1~2)
        # "일반자료실"은 원래 row=1, column=0, rowspan=2 → 이제 row=1, column=1
        ttk.Label(loan_frame, text="일반자료실", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=1, rowspan=2, sticky="nsew", padx=1, pady=1)
        
        ttk.Label(loan_frame, text="금일", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=2, sticky="nsew", padx=1, pady=1)
        ttk.Label(loan_frame, text="누계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=2, column=2, sticky="nsew", padx=1, pady=1)
        
        # 일반자료실 - 금일
        for i, cat in enumerate(categories, start=3):
            key = f"관외_일반_금일_{cat}"
            ent = ttk.Entry(loan_frame, width=6)
            ent.grid(row=1, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # 일반자료실 - 누계
        for i, cat in enumerate(categories, start=3):
            key = f"관외_일반_누계_{cat}"
            ent = ttk.Entry(loan_frame, width=6)
            ent.grid(row=2, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # (2) 어린이자료실: 금일/누계 (행 3~4)
        # "어린이자료실\n(책펴락)" 원래 row=3, column=0, rowspan=2 → 이제 row=3, column=1
        ttk.Label(loan_frame, text="어린이자료실\n(책펴락)", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=3, column=1, rowspan=2, sticky="nsew", padx=1, pady=1)
        
        ttk.Label(loan_frame, text="금일", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=3, column=2, sticky="nsew", padx=1, pady=1)
        ttk.Label(loan_frame, text="누계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=4, column=2, sticky="nsew", padx=1, pady=1)
        
        # 어린이자료실 - 금일
        for i, cat in enumerate(categories, start=3):
            key = f"관외_어린이_금일_{cat}"
            ent = ttk.Entry(loan_frame, width=6)
            ent.grid(row=3, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # 어린이자료실 - 누계
        for i, cat in enumerate(categories, start=3):
            key = f"관외_어린이_누계_{cat}"
            ent = ttk.Entry(loan_frame, width=6)
            ent.grid(row=4, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # (3) 합계: 금일/누계 (행 5~6)
        # "합계" 원래 row=5, column=0, rowspan=2 → 이제 row=5, column=1
        ttk.Label(loan_frame, text="합계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=5, column=1, rowspan=2, sticky="nsew", padx=1, pady=1)
        
        ttk.Label(loan_frame, text="금일", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=5, column=2, sticky="nsew", padx=1, pady=1)
        ttk.Label(loan_frame, text="누계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=6, column=2, sticky="nsew", padx=1, pady=1)
        
        # 합계 - 금일
        for i, cat in enumerate(categories, start=3):
            key = f"관외_합계_금일_{cat}"
            ent = ttk.Entry(loan_frame, width=6)
            ent.grid(row=5, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # 합계 - 누계
        for i, cat in enumerate(categories, start=3):
            key = f"관외_합계_누계_{cat}"
            ent = ttk.Entry(loan_frame, width=6)
            ent.grid(row=6, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # 전체 열의 수는: 1 (제목) + 2 (구분 부분) + len(categories)
        total_columns = 1 + 2 + len(categories)
        for c in range(total_columns):
            loan_frame.columnconfigure(c, weight=1)


    # ─────────────────────────────────────────────────────────────────────
    # (4) 세 번째 표: 자료실 이용자
    # ─────────────────────────────────────────────────────────────────────
    # ─────────────────────────────────────────────────────────────────────
# (4) 세 번째 표: 자료실 이용자
# ─────────────────────────────────────────────────────────────────────
    def create_table3(self):
        new_frame = ttk.Frame(self.container)
        new_frame.pack(fill=tk.X, padx=10, pady=5)
    
        # 열 0: "자료실 이용자" 제목
        ttk.Label(new_frame, text="자료실 이용자", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=0, rowspan=6, sticky="nsew", padx=1, pady=1)
    
        # "구분" 라벨: 열 1, row 0~1 병합
        ttk.Label(new_frame, text="구분", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=1, columnspan=2, rowspan=2, sticky="nsew", padx=1, pady=1)
    
        # 헤더 - 열 4부터
        ttk.Label(new_frame, text="일반자료실", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=4, rowspan=2, sticky="nsew", padx=1, pady=1)
        ttk.Label(new_frame, text="어린이자료실(책펴락)", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=5, rowspan=2, sticky="nsew", padx=1, pady=1)
        ttk.Label(new_frame, text="디지털 코너", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=6, columnspan=3, sticky="nsew", padx=1, pady=1)
        ttk.Label(new_frame, text="합계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=9, rowspan=2, sticky="nsew", padx=1, pady=1)
        ttk.Label(new_frame, text="도서관회원가입", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=10, columnspan=4, sticky="nsew", padx=1, pady=1)
    
        # 디지털 코너 + 회원가입 세부
        ttk.Label(new_frame, text="PC", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=6, sticky="nsew", padx=1, pady=1)
        ttk.Label(new_frame, text="VOD", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=7, sticky="nsew", padx=1, pady=1)
        ttk.Label(new_frame, text="소계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=8, sticky="nsew", padx=1, pady=1)
        ttk.Label(new_frame, text="학생", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=10, sticky="nsew", padx=1, pady=1)
        ttk.Label(new_frame, text="일반", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=11, sticky="nsew", padx=1, pady=1)
        ttk.Label(new_frame, text="합계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=12, sticky="nsew", padx=1, pady=1)
        ttk.Label(new_frame, text="총회원수", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=13, sticky="nsew", padx=1, pady=1)
    
        # ───────────── 열람자 ─────────────
        ttk.Label(new_frame, text="열람자", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=2, column=1, rowspan=2, sticky="nsew", padx=1, pady=1)
        ttk.Label(new_frame, text="금일", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=2, column=2, sticky="nsew", padx=1, pady=1)
        ttk.Label(new_frame, text="누계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=3, column=2, sticky="nsew", padx=1, pady=1)
    
        for c in range(4, 14):
            ent_day = ttk.Entry(new_frame, width=8)
            ent_day.grid(row=2, column=c, sticky="nsew", padx=1, pady=1)
            self.entries[f"자료실_열람자_금일_col{c-3}"] = ent_day
    
            ent_total = ttk.Entry(new_frame, width=8)
            ent_total.grid(row=3, column=c, sticky="nsew", padx=1, pady=1)
            self.entries[f"자료실_열람자_누계_col{c-3}"] = ent_total
    
        # ───────────── 대출자 ─────────────
        ttk.Label(new_frame, text="대출자", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=4, column=1, rowspan=2, sticky="nsew", padx=1, pady=1)
        ttk.Label(new_frame, text="금일", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=4, column=2, sticky="nsew", padx=1, pady=1)
        ttk.Label(new_frame, text="누계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=5, column=2, sticky="nsew", padx=1, pady=1)
    
        for c in range(4, 14):
            ent_day = ttk.Entry(new_frame, width=8)
            ent_day.grid(row=4, column=c, sticky="nsew", padx=1, pady=1)
            self.entries[f"자료실_대출자_금일_col{c-3}"] = ent_day
    
            ent_total = ttk.Entry(new_frame, width=8)
            ent_total.grid(row=5, column=c, sticky="nsew", padx=1, pady=1)
            self.entries[f"자료실_대출자_누계_col{c-3}"] = ent_total
    
        # 추가 항목: 도서관회원가입 이용자수
        ttk.Label(new_frame, text="금일 이용자수", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=4, column=10, columnspan=2, sticky="nsew", padx=1, pady=1)
        ent_day_user = ttk.Entry(new_frame, width=16)
        ent_day_user.grid(row=4, column=12, columnspan=2, sticky="nsew", padx=1, pady=1)
        self.entries["도서관회원가입_금일이용자수"] = ent_day_user
    
        ttk.Label(new_frame, text="총 이용자수", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=5, column=10, columnspan=2, sticky="nsew", padx=1, pady=1)
        ent_total_user = ttk.Entry(new_frame, width=16)
        ent_total_user.grid(row=5, column=12, columnspan=2, sticky="nsew", padx=1, pady=1)
        self.entries["도서관회원가입_총이용자수"] = ent_total_user
    
        # 열 너비 균등 분배
        for c in range(14):
            new_frame.columnconfigure(c, weight=1)






    # ─────────────────────────────────────────────────────────────────────
    # (5) 네 번째 표: 정보봉사 (책바다/책나래)
    # ─────────────────────────────────────────────────────────────────────
    def create_table4(self):
        service_frame = ttk.Frame(self.container)
        service_frame.pack(fill=tk.X, padx=10, pady=5)
    
        # (1) 정보봉사 라벨
        ttk.Label(service_frame, text="정보봉사", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=0, rowspan=2, sticky="nsew", padx=1, pady=1)
    
        # (2) 책바다
        ttk.Label(service_frame, text="책바다", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=1, rowspan=2, sticky="nsew", padx=1, pady=1)
    
        ttk.Label(service_frame, text="금일", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=2, sticky="nsew", padx=1, pady=1)
        self.booksea_day_entry = ttk.Entry(service_frame, width=8)
        self.booksea_day_entry.grid(row=0, column=3, sticky="nsew", padx=1, pady=1)
    
        ttk.Label(service_frame, text="누계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=2, sticky="nsew", padx=1, pady=1)
        self.booksea_total_entry = ttk.Entry(service_frame, width=8)
        self.booksea_total_entry.grid(row=1, column=3, sticky="nsew", padx=1, pady=1)
    
        self.entries["책바다_금일"] = self.booksea_day_entry
        self.entries["책바다_누계"] = self.booksea_total_entry
    
        # (3) 책나래
        ttk.Label(service_frame, text="책나래", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=4, rowspan=2, sticky="nsew", padx=1, pady=1)
    
        ttk.Label(service_frame, text="금일", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=5, sticky="nsew", padx=1, pady=1)
        self.booknara_day_entry = ttk.Entry(service_frame, width=8)
        self.booknara_day_entry.grid(row=0, column=6, sticky="nsew", padx=1, pady=1)
    
        ttk.Label(service_frame, text="누계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=5, sticky="nsew", padx=1, pady=1)
        self.booknara_total_entry = ttk.Entry(service_frame, width=8)
        self.booknara_total_entry.grid(row=1, column=6, sticky="nsew", padx=1, pady=1)
    
        self.entries["책나래_금일"] = self.booknara_day_entry
        self.entries["책나래_누계"] = self.booknara_total_entry
    
        # (4) 열 7 - 금일정보봉사 / 합계 라벨
        ttk.Label(service_frame, text="금일정보봉사", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=7, sticky="nsew", padx=1, pady=1)
        ttk.Label(service_frame, text="합계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=7, sticky="nsew", padx=1, pady=1)
    
        # (5) 열 8 - Entry 두 개 (금일정보 / 합계)
        self.info_day_entry = ttk.Entry(service_frame, width=8, state="readonly")
        self.info_day_entry.grid(row=0, column=8, sticky="nsew", padx=1, pady=1)
    
        self.info_total_entry = ttk.Entry(service_frame, width=8, state="readonly")
        self.info_total_entry.grid(row=1, column=8, sticky="nsew", padx=1, pady=1)
    
        self.entries["정보봉사_금일"] = self.info_day_entry
        self.entries["정보봉사_합계"] = self.info_total_entry
    
        # 자동합계 바인딩
        self.booksea_day_entry.bind("<KeyRelease>", self.update_service_sum)
        self.booksea_total_entry.bind("<KeyRelease>", self.update_service_sum)
        self.booknara_day_entry.bind("<KeyRelease>", self.update_service_sum)
        self.booknara_total_entry.bind("<KeyRelease>", self.update_service_sum)
    
        # 열 크기 설정
        for c in range(9):
            service_frame.columnconfigure(c, weight=1)






    def update_service_sum(self, event=None):
        try:
            booksea_day = float(self.booksea_day_entry.get())
        except ValueError:
            booksea_day = 0.0

        try:
            booknara_day = float(self.booknara_day_entry.get())
        except ValueError:
            booknara_day = 0.0

        day_sum = booksea_day + booknara_day

        try:
            booksea_total = float(self.booksea_total_entry.get())
        except ValueError:
            booksea_total = 0.0

        try:
            booknara_total = float(self.booknara_total_entry.get())
        except ValueError:
            booknara_total = 0.0

        total_sum = booksea_total + booknara_total

        self.info_day_entry.config(state="normal")
        self.info_day_entry.delete(0, tk.END)
        self.info_day_entry.insert(0, str(day_sum))
        self.info_day_entry.config(state="readonly")

        self.info_total_entry.config(state="normal")
        self.info_total_entry.delete(0, tk.END)
        self.info_total_entry.insert(0, str(total_sum))
        self.info_total_entry.config(state="readonly")

    # ─────────────────────────────────────────────────────────────────────
    # (6) 하단 버튼
    # ─────────────────────────────────────────────────────────────────────
    def create_buttons(self):
        btn_frame = ttk.Frame(self.container, padding=10)
        btn_frame.pack()
    
        ttk.Button(btn_frame, text="저장", command=self.save_to_excel).pack(side=tk.LEFT, padx=5)
        ttk.Button(btn_frame, text="Excel 열기", command=self.open_excel).pack(side=tk.LEFT, padx=5)
        ttk.Button(btn_frame, text="초기화", command=self.clear_entries).pack(side=tk.LEFT, padx=5)
        ttk.Button(btn_frame, text="출력", command=self.print_excel).pack(side=tk.LEFT, padx=5)

    # ─────────────────────────────────────────────────────────────────────
    # ★★★ 핵심: openpyxl 로 수정한 save_to_excel() ★★★
    # ─────────────────────────────────────────────────────────────────────
    def save_to_excel(self):
        """
        전체 GUI 데이터를 날짜별 시트에 저장하고,
        날짜 기반 자동 파일명으로 저장 (예: 도서관일지_20250405.xlsx)
        """
        # 날짜 처리
        year = self.year_cmb.get()
        month = self.month_cmb.get().zfill(2)
        day = self.day_cmb.get().zfill(2)
        weekday = self.weekday_cmb.get()
        date_str = f"{year}-{month}-{day} ({weekday}요일)"
        date_short = f"{year}{month}{day}"
        excel_filename = f"도서관일지_{date_short}.xlsx"
    
        # 워크북 생성 또는 열기
        if os.path.exists(excel_filename):
            wb = openpyxl.load_workbook(excel_filename)
        else:
            wb = openpyxl.Workbook()
            wb.remove(wb.active)
        
        # 같은 날짜의 시트가 있다면 삭제 후 새 시트 생성
        if date_str in wb.sheetnames:
            del wb[date_str]
        ws = wb.create_sheet(title=date_str)
    
        # 스타일 공통 세팅
        center = Alignment(horizontal="center", vertical="center", wrap_text=True)
        title_font = Font(size=16, bold=True)
        bold = Font(bold=True)
        border = Border(
            left=Side(style="thin"), right=Side(style="thin"),
            top=Side(style="thin"), bottom=Side(style="thin")
        )
    
        def write_cell(r, c, val, font=None, merge=None):
            cell = ws.cell(row=r, column=c, value=val)
            cell.alignment = center
            cell.font = font or Font()
            cell.border = border
            if merge:
                ws.merge_cells(start_row=r, start_column=c, end_row=merge[0], end_column=merge[1])
    
        # 상단 타이틀 작성
        write_cell(1, 1, "날짜", bold)
        write_cell(1, 2, date_str)
        write_cell(2, 1, "김제학생교육문화관 도서관일지", title_font, merge=(2, 10))
    
        # 각 표를 순차적으로 저장
        next_row = 4
        next_row = self.save_table1_to_excel(ws, next_row)
        next_row = self.save_table2_to_excel(ws, next_row)
        next_row = self.save_table3_to_excel(ws, next_row)
        next_row = self.save_table4_to_excel(ws, next_row)
    
        # 열 너비 일괄 통일 (1~14열까지 똑같이)
        for col in range(1, 15):  # 최대 열 14 기준
            col_letter = openpyxl.utils.get_column_letter(col)
            ws.column_dimensions[col_letter].width = 16  # 원하는 크기로 설정
    
        # 저장 및 메시지
        wb.save(excel_filename)
        messagebox.showinfo("저장 완료", f"'{excel_filename}' 파일에 저장되었습니다.")

    def save_table1_to_excel(self, ws, start_row):
        center = Alignment(horizontal="center", vertical="center", wrap_text=True)
        bold = Font(bold=True)
        border = Border(
            left=Side(style="thin"), right=Side(style="thin"),
            top=Side(style="thin"), bottom=Side(style="thin")
        )
    
        def w(r, c, val, merge=None):
            cell = ws.cell(r, c, val)
            cell.alignment = center
            cell.font = bold
            cell.border = border
            if merge:
                ws.merge_cells(start_row=r, start_column=c, end_row=merge[0], end_column=merge[1])
    
        r = start_row
    
        # (1) 헤더
        w(r, 1, "열람실 이용자", merge=(r+2, 1))
        w(r, 2, "구분")
        w(r, 3, "학생")
        w(r, 4, "일반")
        w(r, 5, "합계")
        w(r, 6, "문화관 교육생", merge=(r+2, 6))
        w(r, 7, "학생교육")
        w(r, 8, "평생교육")
        w(r, 9, "시설견학")
        w(r, 10, "합계")
        w(r, 11, "열람실")
        w(r, 12, "자료실")
    
        # (2) 금일 행
        ws.cell(r+1, 2, "금일").alignment = center
        ws.cell(r+2, 2, "누계").alignment = center
    
        # 금일 열람실
        ws.cell(r+1, 3, self.entries["금일_열람_학생"].get())
        ws.cell(r+1, 4, self.entries["금일_열람_일반"].get())
        ws.cell(r+1, 5, self.entries["금일_열람_합계"].get())
    
        # 누계 열람실
        ws.cell(r+2, 3, self.entries["누계_열람_학생"].get())
        ws.cell(r+2, 4, self.entries["누계_열람_일반"].get())
        ws.cell(r+2, 5, self.entries["누계_열람_합계"].get())
    
        # 금일 문화관
        ws.cell(r+1, 7, self.entries["금일_문화_학생교육"].get())
        ws.cell(r+1, 8, self.entries["금일_문화_평생교육"].get())
        ws.cell(r+1, 9, self.entries["금일_문화_시설견학"].get())
        ws.cell(r+1, 10, self.entries["금일_문화_합계"].get())
    
        # 누계 문화관
        ws.cell(r+2, 7, self.entries["누계_문화_학생교육"].get())
        ws.cell(r+2, 8, self.entries["누계_문화_평생교육"].get())
        ws.cell(r+2, 9, self.entries["누계_문화_시설견학"].get())
        ws.cell(r+2, 10, self.entries["누계_문화_합계"].get())
    
        # 개관일수
        ws.cell(r+1, 11, self.entries["금일_개관_열람실"].get())
        ws.cell(r+2, 12, self.entries["누계_개관_자료실"].get())
    
        # 스타일 전체 적용
        for i in range(r, r+3):
            for j in range(1, 13):
                cell = ws.cell(i, j)
                cell.alignment = center
                cell.border = border
                
        for i in range(r, r+2):  # 표4는 2행이니까
            for j in range(10, 15):
                ws.cell(i, j, "").alignment = center
                ws.cell(i, j).border = border
        
        return r + 4  # 다음 표가 시작할 row 반환

    
    def save_table2_to_excel(self, ws, start_row):
        center = Alignment(horizontal="center", vertical="center", wrap_text=True)
        bold = Font(bold=True)
        border = Border(
            left=Side(style="thin"), right=Side(style="thin"),
            top=Side(style="thin"), bottom=Side(style="thin")
        )
    
        def w(r, c, val, merge=None):
            cell = ws.cell(r, c, val)
            cell.alignment = center
            cell.font = bold
            cell.border = border
            if merge:
                ws.merge_cells(start_row=r, start_column=c, end_row=merge[0], end_column=merge[1])
    
        categories = ["총류", "철학", "종교", "사회과학", "자연과학",
                      "기술과학", "예술", "언어", "문학", "역사", "계"]
    
        r = start_row
    
        # 제목 셀
        w(r, 1, "관외 대출 도서", merge=(r+6, 1))
        w(r, 2, "구분", merge=(r, 3))
        for i, cat in enumerate(categories, start=4):
            w(r, i, cat)
    
        row_map = {
            "일반자료실": r+1,
            "어린이자료실": r+3,
            "합계": r+5
        }
    
        for label, base_row in row_map.items():
            # 구분 라벨
            w(base_row, 2, label, merge=(base_row+1, 2))
            ws.cell(base_row, 3, "금일").alignment = center
            ws.cell(base_row+1, 3, "누계").alignment = center
    
            for i, cat in enumerate(categories, start=4):
                key_day = f"관외_{label.replace('(', '').replace(')', '').replace('\\n', '')}_금일_{cat}"
                key_total = f"관외_{label.replace('(', '').replace(')', '').replace('\\n', '')}_누계_{cat}"
    
                val_day = self.entries.get(key_day, tk.StringVar()).get()
                val_total = self.entries.get(key_total, tk.StringVar()).get()
    
                ws.cell(base_row, i, val_day).alignment = center
                ws.cell(base_row+1, i, val_total).alignment = center
    
        # 전체 셀 스타일
        for i in range(r, r+7):
            for j in range(1, 4 + len(categories)):
                ws.cell(i, j).border = border
                ws.cell(i, j).alignment = center

        for i in range(r, r+2):  # 표4는 2행이니까
            for j in range(10, 15):
                ws.cell(i, j, "").alignment = center
                ws.cell(i, j).border = border
    
        return r + 8

    def save_table3_to_excel(self, ws, start_row):
        # 기본 스타일 설정
        center = Alignment(horizontal="center", vertical="center", wrap_text=True)
        border = Border(
            left=Side(style='thin'),
            right=Side(style='thin'),
            top=Side(style='thin'),
            bottom=Side(style='thin')
        )
        r = start_row
    
        # 1. "자료실 이용자" (세로 6행 병합, 열 1)
        ws.merge_cells(start_row=r, start_column=1, end_row=r+5, end_column=1)
        ws.cell(row=r, column=1, value="자료실 이용자")
    
        # 2. "구분" (세로 2행 병합, 열 2~3)
        ws.merge_cells(start_row=r, start_column=2, end_row=r+1, end_column=3)
        ws.cell(row=r, column=2, value="구분")
    
        # 3. 헤더 설정
        # 그룹 A: vertical merge (row 0~1) – 서브헤더가 없음
        #    - "일반자료실" (열 4, colspan 1)
        #    - "어린이자료실(책펴락)" (열 5, colspan 1)
        #    - "합계" (열 9, colspan 1)
        groupA = [("일반자료실", 4, 1), ("어린이자료실(책펴락)", 5, 1), ("합계", 9, 1)]
        for text, col, colspan in groupA:
            ws.merge_cells(start_row=r, start_column=col, end_row=r+1, end_column=col+colspan-1)
            ws.cell(row=r, column=col, value=text)
    
        # 그룹 B: horizontal merge만 – 서브헤더가 아래에 들어감
        #    - "디지털 코너" (열 6, colspan 3)
        #    - "도서관회원가입" (열 10, colspan 4)
        groupB = [("디지털 코너", 6, 3), ("도서관회원가입", 10, 4)]
        for text, col, colspan in groupB:
            if colspan > 1:
                ws.merge_cells(start_row=r, start_column=col, end_row=r, end_column=col+colspan-1)
            ws.cell(row=r, column=col, value=text)
    
        # 4. 그룹 B의 서브헤더 작성 (row r+1)
        # "디지털 코너"의 서브헤더: "PC", "VOD", "소계" (열 6,7,8)
        # "도서관회원가입"의 서브헤더: "학생", "일반", "합계", "총회원수" (열 10,11,12,13)
        subheaders_B = {6: ["PC", "VOD", "소계"],
                        10: ["학생", "일반", "합계", "총회원수"]}
        for start_col, subs in subheaders_B.items():
            for i, sub in enumerate(subs):
                ws.cell(row=r+1, column=start_col+i, value=sub)
    
        # 5. "열람자" 영역 (row r+2~r+3)
        ws.merge_cells(start_row=r+2, start_column=2, end_row=r+3, end_column=2)
        ws.cell(row=r+2, column=2, value="열람자")
        ws.cell(row=r+2, column=3, value="금일")
        ws.cell(row=r+3, column=3, value="누계")
        for c in range(4, 14):
            col_idx = c - 3
            ws.cell(row=r+2, column=c, value=self.entries[f"자료실_열람자_금일_col{col_idx}"].get())
            ws.cell(row=r+3, column=c, value=self.entries[f"자료실_열람자_누계_col{col_idx}"].get())
    
        # 6. "대출자" 영역 (row r+4~r+5)
        ws.merge_cells(start_row=r+4, start_column=2, end_row=r+5, end_column=2)
        ws.cell(row=r+4, column=2, value="대출자")
        ws.cell(row=r+4, column=3, value="금일")
        ws.cell(row=r+5, column=3, value="누계")
        for c in range(4, 14):
            col_idx = c - 3
            ws.cell(row=r+4, column=c, value=self.entries[f"자료실_대출자_금일_col{col_idx}"].get())
            ws.cell(row=r+5, column=c, value=self.entries[f"자료실_대출자_누계_col{col_idx}"].get())
    
        # 7. "도서관회원가입" 하단 추가 항목
        # Row r+4: "금일 이용자수" – 10~11, 값은 12~13
        ws.merge_cells(start_row=r+4, start_column=10, end_row=r+4, end_column=11)
        ws.cell(row=r+4, column=10, value="금일 이용자수")
        ws.merge_cells(start_row=r+4, start_column=12, end_row=r+4, end_column=13)
        ws.cell(row=r+4, column=12, value=self.entries["도서관회원가입_금일이용자수"].get())
        # Row r+5: "총 이용자수" – 10~11, 값은 12~13
        ws.merge_cells(start_row=r+5, start_column=10, end_row=r+5, end_column=11)
        ws.cell(row=r+5, column=10, value="총 이용자수")
        ws.merge_cells(start_row=r+5, start_column=12, end_row=r+5, end_column=13)
        ws.cell(row=r+5, column=12, value=self.entries["도서관회원가입_총이용자수"].get())
    
        # 8. 스타일 적용
        from openpyxl.cell.cell import MergedCell
        for i in range(r, r+6):
            for j in range(1, 14):
                cell = ws.cell(i, j)
                # 병합된 영역의 시작 셀이 아니면 MergedCell 객체가 반환되므로 건너뜁니다.
                if not isinstance(cell, MergedCell):
                    cell.alignment = center
                    cell.border = border
    
        return r + 6

    def save_table4_to_excel(self, ws, start_row):
        center = Alignment(horizontal="center", vertical="center", wrap_text=True)
        bold = Font(bold=True)
        border = Border(
            left=Side(style="thin"), right=Side(style="thin"),
            top=Side(style="thin"), bottom=Side(style="thin")
        )
    
        def w(r, c, val, merge=None):
            cell = ws.cell(r, c, val)
            cell.alignment = center
            cell.font = bold
            cell.border = border
            if merge:
                ws.merge_cells(start_row=r, start_column=c, end_row=merge[0], end_column=merge[1])
    
        r = start_row
    
        # 정보봉사 / 책바다 / 책나래
        w(r, 1, "정보봉사", merge=(r+1, 1))
        w(r, 2, "책바다", merge=(r+1, 2))
        ws.cell(r, 3, "금일").alignment = center
        ws.cell(r+1, 3, "누계").alignment = center
        ws.cell(r, 3).border = border
        ws.cell(r+1, 3).border = border
    
        ws.cell(r, 4, self.entries["책바다_금일"].get()).alignment = center
        ws.cell(r+1, 4, self.entries["책바다_누계"].get()).alignment = center
    
        w(r, 5, "책나래", merge=(r+1, 5))
        ws.cell(r, 6, "금일").alignment = center
        ws.cell(r+1, 6, "누계").alignment = center
        ws.cell(r, 6).border = border
        ws.cell(r+1, 6).border = border
    
        ws.cell(r, 7, self.entries["책나래_금일"].get()).alignment = center
        ws.cell(r+1, 7, self.entries["책나래_누계"].get()).alignment = center
    
        # 정보봉사 합계 영역
        ws.cell(r, 8, "금일정보봉사").alignment = center
        ws.cell(r+1, 8, "합계").alignment = center
        ws.cell(r, 8).border = border
        ws.cell(r+1, 8).border = border
    
        ws.cell(r, 9, self.entries["정보봉사_금일"].get()).alignment = center
        ws.cell(r+1, 9, self.entries["정보봉사_합계"].get()).alignment = center
    
        # 모든 셀 테두리 적용
        for i in range(r, r+2):
            for j in range(1, 10):
                ws.cell(i, j).border = border
                ws.cell(i, j).alignment = center

        for i in range(r, r+2):  # 표4는 2행이니까
            for j in range(10, 15):
                ws.cell(i, j, "").alignment = center
                ws.cell(i, j).border = border
    
        return r + 3



    # ─────────────────────────────────────────────────────────────────────
    def open_excel(self):
        """현재 날짜 기반 파일 열기"""
        file_name = self.get_excel_filename()
        if not os.path.exists(file_name):
            messagebox.showwarning("파일 없음", f"'{file_name}' 파일이 없습니다.")
            return
        try:
            os.startfile(os.path.abspath(file_name))
        except Exception as e:
            messagebox.showerror("오류", f"파일을 열 수 없습니다:\n{str(e)}")

    def clear_entries(self):
        """모든 입력 필드 초기화 및 날짜 콤보박스 현재 날짜로 재설정"""
        for entry in self.entries.values():
            entry.delete(0, tk.END)
    
        # 날짜 초기화 (오늘 날짜)
        now = datetime.now()
        self.year_cmb.set(str(now.year))
        self.month_cmb.set(str(now.month))
        self.day_cmb.set(str(now.day))
        self.weekday_cmb.set(self.weekdays[now.weekday() % 7])

    def print_excel(self):
        """현재 날짜 기반 Excel 파일을 프린터로 출력"""
        file_name = self.get_excel_filename()
        if not os.path.exists(file_name):
            messagebox.showwarning("파일 없음", f"'{file_name}' 파일이 없습니다.")
            return
        try:
            excel = win32.Dispatch("Excel.Application")
            wb = excel.Workbooks.Open(os.path.abspath(file_name))
            ws = wb.Worksheets(1)
            ws.PrintOut()
            wb.Close(False)
            excel.Quit()
            messagebox.showinfo("출력 완료", f"'{file_name}' 출력 완료!")
        except Exception as e:
            messagebox.showerror("인쇄 오류", str(e))

    def get_excel_filename(self):
        """현재 날짜 기반 파일명 반환"""
        y = self.year_cmb.get()
        m = self.month_cmb.get().zfill(2)
        d = self.day_cmb.get().zfill(2)
        return f"도서관일지_{y}{m}{d}.xlsx"

# 메인 구동부
if __name__ == "__main__":
    root = tk.Tk()
    app = LibraryApp(root)
    root.mainloop()
